In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time

from torch.utils.data import TensorDataset, DataLoader

from llama import Tokenizer
tokenizer = Tokenizer('./tokenizer.model')

#hyperparameters
n_embd = 256
block_size = 128
n_head = 4
n_layer = 4
vocab_size = 32_000
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-3

text = open('../data/shakespeare.txt', encoding="utf-8").read()

data = torch.tensor(tokenizer.encode(text, False, False))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

def get_all_data(split):
    
    data = train_data if split == "train" else val_data
    X = torch.stack([data[i:i+block_size] for i in range(len(data) - block_size)])
    y = torch.stack([data[i+1:i+1+block_size] for i in range(len(data) - block_size)])
    X, y = X.to(device), y.to(device)
    return X, y


X, y = get_all_data('train')
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        
    def forward(self, x):
        
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x
    
class FeedForward(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(n_embd * 4, n_embd)
        )
        
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        
        head_size = n_embd // n_head
        
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        
        x = x + self.ln1(self.sa(x))
        x = x + self.ln2(self.ffwd(x))
        
        return x
    
class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
        else:
            
            B,T,C = logits.shape
            
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            
            # Create a mask where targets are not equal to -1 (pad_id)
            mask = targets != -1
            # Use this mask to filter out logits and targets that you don't want to consider for the loss
            logits_masked = logits[mask]
            targets_masked = targets[mask]

            # Calculate loss with the non-padded targets and logits
            loss = F.cross_entropy(logits_masked, targets_masked)
            
        return logits, loss
    
    def generate(self, idx, max_gen_len):
        
        initial_idx = idx
        
        out = torch.zeros(max_gen_len).to(device)
        
        for i in range(max_gen_len):
            
            idx_cond = idx[:, -block_size:]
            
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :]
            
            probs = F.softmax(logits, dim=-1)
            
            pred = torch.multinomial(probs, 1)
            
            out[i] = pred.item()
            
            idx = torch.cat((idx, pred), dim=-1)
            
        out = torch.cat((initial_idx.squeeze(0), out), dim=-1)
            
        print(tokenizer.decode(out.long().tolist()))
            
    
        
        
    
model = Transformer().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)



In [86]:
xb = torch.tensor(tokenizer.encode('The best person is ', False, False))
xb = xb.to(device)

yb = torch.tensor(tokenizer.encode('Clark', False, False))
pad_size = len(xb) - len(yb)
yb_padded = torch.cat([torch.tensor([tokenizer.pad_id] * pad_size), yb])
yb = yb_padded.long().to(device)

count = 0
epochs = 10

for epoch in range(epochs):
    
    logits, loss = model(xb.unsqueeze(0), yb)
    
#     if count % (epochs // 10) == 0:
    print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    count += 1
    if count == epochs:
        break

tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)


In [92]:
model.generate(idx=torch.tensor(tokenizer.encode('The best person is ', False, False)).unsqueeze(0).to(device), max_gen_len=1)

The best person is  Clark


In [95]:
model.generate(idx=torch.tensor(tokenizer.encode('HERMIONE: The best person is ', False, False)).unsqueeze(0).to(device), max_gen_len=500)

HERMIONE: The best person is  Clark, there,
Her lord.

RIARWould understand any thing, thou partner belong to be pardon me, at which you--
LAND:

VOLUMNIA:
Moreaught not my wife.
his brother.

Provost sound?

Then go not speak; give Paris seem'sake,
And know not:
No, in.

KING HEN:
GLOUCESTER:
We must dinner our special up,
The time that'er; or in!
Which of trust us minose top inexely nephew, blemises am the breaking, you not moving so well understood.

RIARWas, he have desperate sorrow;
And that you canby Place I' soldiers the worth boon accident that God!

Give degrees Citizen:
My Lord:
I will take this place seen me, the more.

KING RICHARD:
That little without yoke to churchmen?

My dangerous treacherous hands in the king.

CLIFFORD: and tie punishment again! a brain of my foes two weakness that love,
I fear, he wore his concludConck of thee more choice:
He were more forecast.

K:
Or slaughter,
For that liege, such thoughts that I from poor prisoner, i inst goes that stands general

In [89]:
count = 0
epochs = 500

for xb, yb in dataloader:
    
    logits, loss = model(xb, yb)
    
    if count % (epochs // 10) == 0:
        print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    count += 1
    if count == epochs:
        break

tensor(4.5463, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0693, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.9347, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.9010, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.8507, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7608, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.8157, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7451, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7601, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.8533, device='cuda:0', grad_fn=<NllLossBackward0>)
